# U-Net Depth (Projeto de dissertação)

In [1]:
running_on_kaggle = False
running_on_colab = True
running_remote = running_on_kaggle or running_on_colab
db_name = 'unet_depth'
repository_name = 'unet_depth'

## Clonar e navegar para o repositório

In [ ]:
if running_on_colab:
    from google.colab import drive

    drive.mount('/content/drive')

In [2]:
import os

# Clone e navegue para o repositório da dissertação
if running_remote:
    !git clone "https://github.com/duraes-antonio/unet_depth"
    !cd $repository_name
    os.chdir(repository_name)

In [3]:
if running_remote:
    !git checkout poc

## Definir variáveis de ambiente e credenciais para serviços

In [ ]:
if running_remote:
    !pip install pymongo[srv] dnspython keras-unet-collection python-dotenv imutils
    !pip install py-cpuinfo gpuinfo

In [4]:
import ast
import json

if running_remote:
    if running_on_kaggle:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        database_url = user_secrets.get_secret("DATABASE_URL")
        google_cred_json = ast.literal_eval(user_secrets.get_secret("GOOGLE_CREDENTIALS_JSON"))
        google_token_json = ast.literal_eval(user_secrets.get_secret("GOOGLE_TOKEN_JSON"))
        os.environ["DATABASE_URL"] = database_url
        os.environ["DB_NAME"] = db_name

    else:
        import dotenv

        path_auth = '../drive/MyDrive/unet_depth/auth/.env'
        dotenv.load_dotenv(path_auth)
        database_url = os.environ["DATABASE_URL"]
        google_cred_json = ast.literal_eval(os.environ["GOOGLE_CREDENTIALS_JSON"])
        google_token_json = ast.literal_eval(os.environ["GOOGLE_TOKEN_JSON"])

    with open('google_credentials.json', 'w', encoding='utf-8') as f:
        json.dump(google_cred_json, f, ensure_ascii=False, indent=4)

    with open('token.json', 'w', encoding='utf-8') as f:
        json.dump(google_token_json, f, ensure_ascii=False, indent=4)

## Instanciar serviços para persistência de resultados e blob

In [5]:
from infra.services.results_service_mongodb import ResultServiceMongoDB
from domain.services.results_service import ResultService
from infra.services.blob_storage.blob_storage_service_google_drive import GoogleDriveBlobStorageService
from infra.services.model_storage_service_google_drive import ModelStorageServiceGoogleDrive
from domain.services.model_storage_service import ModelStorageService
from infra.services.test_case_execution_service_mongodb import TestCaseExecutionServiceMongoDB
from infra.services.test_case_service_mongodb import TestCaseServiceMongoDB
from domain.services.test_case_execution_service import TestCaseExecutionService
from domain.services.blob_storage_service import BlobStorageService
from domain.services.test_case_service import TestCaseService

test_case_serv: TestCaseService = TestCaseServiceMongoDB(db_name)
execution_serv: TestCaseExecutionService = TestCaseExecutionServiceMongoDB(db_name)

blob_service: BlobStorageService = GoogleDriveBlobStorageService()
model_storage: ModelStorageService = ModelStorageServiceGoogleDrive(blob_service)

result_service: ResultService = ResultServiceMongoDB(db_name)

## Baixar dataset

In [6]:
if not os.path.exists('data'):
    !git clone "https://gitlab.com/siddinc/new_depth.git" "./data"

## Executar aplicação

In [ ]:
import keras
from infra.util.dataset import read_nyu_csv
from infra.util.output import plot_image_comparison
from infra.application_manager import ApplicationManager

train_path = "./data/nyu2_train.csv"
test_path = "./data/nyu2_test.csv"
batch_size = 8
image_size = 256, 256
epochs = 40

# Defina a semente usada em operações como embaralhamento do dataset
seed = 42


# callback após avaliação em base de teste
def after_test(model: keras.Model):
    xy_path_pairs = read_nyu_csv(test_path)
    return plot_image_comparison(model, xy_path_pairs)


try:
    application = ApplicationManager(
        blob_service, model_storage, execution_serv,
        test_case_serv, result_service, image_size, epochs
    )
    application.prepare_train_data(train_path, batch_size, True, seed, 0.1)
    application.prepare_test_data(test_path, batch_size, True, seed)
    application.run()

except Exception as E:
    print(E)


----------PROCESSANDO CASO DE TESTE----------
ID:             63556b47635f33b1d2b770f8
Network:        Networks.AttentionUNet
Backbone:       KerasBackbone.VGG16
Otimizador:     Optimizers.Adam
Pesos imagenet: True
---------------------------------------------
    
Caso de teste marcado como ocupado! ID: 63556b47635f33b1d2b770f8
Última execução do casos de teste: None
Iniciado: build do modelo


C:\Users\AEVO\Desktop\pessoal\IFES\dissertacao\unet_depth\venv\lib\site-packages\keras_unet_collection\_backbone_zoo.py:45: UserWarning: 

Backbone VGG16 does not use batch norm, but other layers received batch_norm=True
  warnings.warn(param_mismatch);


Finalizado: build do modelo
Iniciado: compilação do modelo
Finalizado compilação do modelo
Iniciado: treinamento
Epoch 1/40
